# Alaskan weather conditions

## Motivation

My partner currently works for an NPR affiliated station in Alaska. Every hour, a member of the station manually aggregates weather information from several websites. If this task takes 5 minutes every hour, that means a member of the station is spending at least 40 minutes every day, over 3 hours every work week, or over 6 days every year copying and pasting information.

Luckily, we can use the Weather Underground API to automate most of this process, and the National Weather Service has a plaintext marine forecast link for the remainder of the information.

## Setup

First, I'm going to import necessary packages, configure the base URL for the Weather Underground API, and define a list of locations and latlon coordinates to reference for requests.

In [1]:
import datetime
import json
import re
import requests

import pytz


_URL_BASE = 'http://api.wunderground.com/api/{key}/{feature}/q/{query}.json'

_LATLONS = {'anchor_point': '59.7775,-151.7702',
            'anchorage': '61.2167,-149.9000',
            'cooper_landing': '60.4905,-149.7944',
            'homer': '59.6425,-152.5483',
            'kachemak_bay_seldovia': '59.4394,-151.7122',
            'kenai': '60.5586,-151.2297',
            'kenai_river': '60.5439,-151.2786',
            'nanwalek': '59.3536,-151.9125',
            'ninilchik': '60.04638,-151.6672',
            'port_graham': '59.3477,-151.8333',
            'soldotna': '60.4866,-151.07527',
            'western_kenai_peninsula': '59.8861,-151.6338'
            }

## Sunrise, sunset, and total daylight

Now, I can request and format the daylight information, being careful about timezones.

In [3]:
_STRING_DAYLIGHT = """
Daylight
  Sunrise:  {sunrise_hour:0>2d}:{sunrise_minute:0>2d}
  Sunset:   {sunset_hour:0>2d}:{sunset_minute:0>2d}
  Total:    {total_hours} hours and {total_minutes} minutes
"""


def get_total_daylight():
    sunrise_hour, sunrise_min, sunset_hour, sunset_min = _get_sun_phase_data_from_weather_underground()
    sunrise = _get_datetime_object_for_sun_phase(sunrise_hour, sunrise_min)
    sunset = _get_datetime_object_for_sun_phase(sunset_hour, sunset_min)
    daylight_hours, daylight_minutes = _get_total_daylight_hours_and_minutes(sunset - sunrise)
    return _STRING_DAYLIGHT.format(
        sunrise_hour=sunrise.hour,
        sunrise_minute=sunrise.minute,
        sunset_hour=sunset.hour,
        sunset_minute=sunset.minute,
        total_hours=daylight_hours,
        total_minutes=daylight_minutes
        )


def _get_sun_phase_data_from_weather_underground():
    url = _URL_BASE.format(key=_KEY, feature='astronomy', query=_LATLONS['homer'])
    data = json.loads(requests.get(url).text)
    data_sunrise = data['sun_phase']['sunrise']
    sunrise_hour = int(data_sunrise['hour'])
    sunrise_min = int(data_sunrise['minute'])
    data_sunset = data['sun_phase']['sunset']
    sunset_hour = int(data_sunset['hour'])
    sunset_min = int(data_sunset['minute'])
    return (sunrise_hour, sunrise_min, sunset_hour, sunset_min)


def _get_datetime_object_for_sun_phase(phase_hour, phase_min):
    year, month, day = _get_datetime_in_alaska_tz()
    return datetime.datetime(year, month, day, phase_hour, phase_min)


def _get_datetime_in_alaska_tz():
    timezone_utc = pytz.timezone('UTC')
    timezone_ak = pytz.timezone('US/Alaska')
    datetime_utc = datetime.datetime.utcnow().replace(tzinfo=timezone_utc)
    datetime_ak = datetime_utc.astimezone(timezone_ak)
    return (datetime_ak.year, datetime_ak.month, datetime_ak.day)


def _get_total_daylight_hours_and_minutes(datetime_delta):
    daylight_raw_minutes = datetime_delta.total_seconds() / 60
    daylight_hours = int(daylight_raw_minutes / 60)
    daylight_minutes = int(daylight_raw_minutes % 60)
    return (daylight_hours, daylight_minutes)

In [4]:
daylight = get_total_daylight()
print(daylight)


Daylight
  Sunrise:  06:25
  Sunset:   22:02
  Total:    15 hours and 37 minutes



## High and low tides

The next piece of information we care about is tidal heights.

In [5]:
_KEY_TIDE_HIGH = 'High Tide'
_KEY_TIDE_LOW = 'Low Tide'
_KEY_HOUR = 'hour'
_KEY_MINUTE = 'minute'
_KEY_TIDE_HEIGHT = 'height'

_STRING_TIDES = """
Tides
  Kachemak Bay, Seldovia
{kachemak_bay_seldovia_detail}
  Kenai River
{kenai_river_detail}
"""

_STRING_TIDES_DETAIL = """    {month:0>2d}/{day:0>2d}
      High tide:  {high_hour}:{high_minute} ({high_height})
      Low tide:   {low_hour}:{low_minute} ({low_height})
"""


def get_tides():
    key_locations = ['kachemak_bay_seldovia', 'kenai_river']
    tide_string_details = {}
    for key_location in key_locations:
        raw_data_tides = _get_tide_data_from_weather_underground(key_location)
        data_tides = _format_tide_data(raw_data_tides)
        location_details = _format_tide_detail_strings(data_tides)
        tide_string_details[key_location + '_detail'] = location_details
    return _STRING_TIDES.format(**tide_string_details)


def _get_tide_data_from_weather_underground(key_location):
    url = _URL_BASE.format(key=_KEY, feature='tide', query=_LATLONS[key_location])
    return json.loads(requests.get(url).text)


def _format_tide_data(raw_data_tides):
    data_tides = {}
    for datum in raw_data_tides['tide']['tideSummary']:
        if datum['data']['type'] not in [_KEY_TIDE_HIGH, _KEY_TIDE_LOW]:
            continue
        date = datum['date']
        datetime_ak = datetime.datetime(int(date['year']), int(date['mon']), int(date['mday']))
        data_tides.setdefault(datetime_ak, {})[datum['data']['type']] = \
            {_KEY_HOUR: date['hour'],
             _KEY_MINUTE: date['min'],
             _KEY_TIDE_HEIGHT: datum['data']['height']}
    return data_tides


def _format_tide_detail_strings(data_tides):
    tide_string_details = []
    for datetime_ak, datum in sorted(data_tides.items(), key=lambda dt: dt[0]):
        tide_high = datum.get(_KEY_TIDE_HIGH, {})
        tide_low = datum.get(_KEY_TIDE_LOW, {})
        tide_string_details.append(
            _STRING_TIDES_DETAIL.format(
                month=datetime_ak.month, 
                day=datetime_ak.day,
                high_hour=tide_high.get(_KEY_HOUR, 'XX'),
                high_minute=tide_high.get(_KEY_MINUTE, 'XX'),
                high_height=tide_high.get(_KEY_TIDE_HEIGHT, 'XX'),
                low_hour=tide_low.get(_KEY_HOUR, 'XX'),
                low_minute=tide_low.get(_KEY_MINUTE, 'XX'),
                low_height=tide_low.get(_KEY_TIDE_HEIGHT, 'XX')
                ))
    return ''.join(tide_string_details)

In [6]:
tides = get_tides()
print(tides)


Tides
  Kachemak Bay, Seldovia
    08/14
      High tide:  13:08 (14.03 ft)
      Low tide:   18:36 (5.86 ft)
    08/15
      High tide:  13:51 (15.46 ft)
      Low tide:   19:25 (4.45 ft)
    08/16
      High tide:  14:29 (16.92 ft)
      Low tide:   20:09 (2.88 ft)
    08/17
      High tide:  15:05 (18.27 ft)
      Low tide:   20:49 (1.35 ft)
    08/18
      High tide:  02:53 (19.94 ft)
      Low tide:   09:15 (-2.99 ft)

  Kenai River
    08/14
      High tide:  15:00 (16.73 ft)
      Low tide:   20:54 (6.36 ft)
    08/15
      High tide:  15:43 (18.16 ft)
      Low tide:   21:43 (4.95 ft)
    08/16
      High tide:  16:21 (19.62 ft)
      Low tide:   22:27 (3.38 ft)
    08/17
      High tide:  16:57 (20.97 ft)
      Low tide:   23:07 (1.85 ft)
    08/18
      High tide:  04:45 (22.64 ft)
      Low tide:   XX:XX (XX)




## Current temperatures

We also need current temperatures for several communities in the area.

In [7]:
_STRING_CURRENT_TEMPERATURE = """
Current temperature
{current_temperature_detail}
"""

_STRING_CURRENT_TEMPERATURE_DETAIL = """  {city:<{padding}s}  {temperature:.0f}
"""

_ORDER_CURRENT_TEMPERATURE = [
    ('homer', 'Homer'),
    ('anchor_point', 'Anchor Point'),
    ('ninilchik', 'Ninilchik'),
    ('kachemak_bay_seldovia', 'Seldovia'),
    ('port_graham', 'Port Graham'),
    ('nanwalek', 'Nanwalek'),
    ('kenai', 'Kenai'),
    ('soldotna', 'Soldotna'),
    ('cooper_landing', 'Cooper Landing'),
    ('anchorage', 'Anchorage')
    ]


def get_current_temperature():
    string_padding = max(len(city) for _, city in _ORDER_CURRENT_TEMPERATURE)
    string_temperature_details = []
    for key, city in _ORDER_CURRENT_TEMPERATURE:
        raw_temp = _get_current_temperature_data_from_weather_underground(key)
        city_detail = _STRING_CURRENT_TEMPERATURE_DETAIL.format(
            city=city + ':', padding=string_padding, temperature=raw_temp)
        string_temperature_details.append(city_detail)
    return _STRING_CURRENT_TEMPERATURE.format(current_temperature_detail=''.join(string_temperature_details))


def _get_current_temperature_data_from_weather_underground(key_location):
    url = _URL_BASE.format(key=_KEY, feature='conditions', query=_LATLONS[key_location])
    return json.loads(requests.get(url).text)['current_observation']['temp_f']

In [8]:
current_temperatures = get_current_temperature()
print(current_temperatures)


Current temperature
  Homer:          53
  Anchor Point:   53
  Ninilchik:      53
  Seldovia:       50
  Port Graham:    50




## Descriptive forecasts

The last pieces of information we want from Weather Underground are the descriptive forecasts for two regions.

In [9]:
_STRING_FORECAST = """
Forecast
  Western Kenai Peninsula
{western_kenai_peninsula_detail}
  Anchorage
{anchorage_detail}
"""

_STRING_FORECAST_DETAIL = """    {title}:  {forecast}
"""

_ORDER_FORECAST = [
    ('western_kenai_peninsula', 'Western Kenai Peninsula'),
    ('anchorage', 'Anchorage')
    ]


def get_forecast():
    location_details = {}
    for key, name in _ORDER_FORECAST:
        raw_data_forecast = _get_forecast_data_from_weather_underground(key)
        data_forecast = _format_forecast_data(raw_data_forecast)
        location_details[key + '_detail'] = ''.join(
            _STRING_FORECAST_DETAIL.format(title=title, forecast=forecast)
            for title, forecast in data_forecast)
    return _STRING_FORECAST.format(**location_details)


def _get_forecast_data_from_weather_underground(key_location):
    url = _URL_BASE.format(key=_KEY, feature='forecast', query=_LATLONS[key_location])
    return json.loads(requests.get(url).text)


def _format_forecast_data(raw_data_forecast):
    data_forecast = []
    for datum in raw_data_forecast['forecast']['txt_forecast']['forecastday']:
        if datum['period'] == 4:
            break
        data_forecast.append((datum['title'], datum['fcttext']))
    return data_forecast

In [10]:
forecast = get_forecast()
print(forecast)


Forecast
  Western Kenai Peninsula
    Sunday:  Showers this morning, becoming a steady rain during the afternoon hours. High near 60F. Winds NNE at 5 to 10 mph. Chance of rain 80%. Rainfall near a half an inch.
    Sunday Night:  Steady light rain this evening. Showers continuing overnight. Low 49F. Winds light and variable. Chance of rain 70%.
    Monday:  Rain showers in the morning will evolve into a more steady rain in the afternoon. High around 60F. Winds light and variable. Chance of rain 60%.
    Monday Night:  Partly cloudy. Low 48F. Winds SSW at 5 to 10 mph.

  Anchorage
    Sunday:  Rain. High 62F. Winds E at 5 to 10 mph. Chance of rain 70%. Rainfall around a quarter of an inch.
    Sunday Night:  Cloudy with occasional rain showers. Low 52F. Winds ESE at 5 to 10 mph. Chance of rain 50%.
    Monday:  Rain showers early with some sunshine later in the day. High 63F. Winds NNE at 5 to 10 mph. Chance of rain 40%.
    Monday Night:  Rain showers early with clearing later at nig

## Descriptive marine forecasts

Finally, we can switch gears and grab the descriptive marine forecasts for several locations from the National Weather Service

In [26]:
_URL_MARINE_FORECAST = 'http://tgftp.nws.noaa.gov/data/raw/fz/fzak51.pafc.cwf.aer.txt'
_SEPARATOR_MARINE_FORECAST = '\$\$'

_ORDER_MARINE_FORECAST = [
    'KACHEMAK BAY',
    'COOK INLET NORTH OF KALGIN ISLAND',
    'COOK INLET KALGIN ISLAND TO POINT BEDE',
    'SHELIKOF STRAIT',
    'BARREN ISLANDS EAST',
    'WEST OF BARREN ISLANDS INCLUDING KAMISHAK BAY',
    'CAPE CLEARE TO GORE POINT',
]

In [28]:
def get_marine_forecast():
    # Get and split the raw text into forecasts for individual locations
    raw_text = requests.get(_URL_MARINE_FORECAST).text
    split_text = re.split(_SEPARATOR_MARINE_FORECAST, raw_text)
    # Sort forecasts for select locations
    ordered_forecasts = []
    for location in _ORDER_MARINE_FORECAST:
        idx_found = [bool(re.search(location, forecast))
                     for forecast in split_text].index(True)
        ordered_forecasts.append(split_text[idx_found])
    return ''.join(ordered_forecasts)

In [31]:
marine_forecasts = get_marine_forecast()
print(marine_forecasts)



PKZ141-220100-
KACHEMAK BAY-
352 AM AKDT SUN AUG 21 2016

...SMALL CRAFT ADVISORY THROUGH TONIGHT...

.TODAY...S WIND 15 KT BECOMING SE 25 KT IN THE AFTERNOON...STRONGEST
ALONG THE OUTER BAY. SEAS 2 FT BUILDING TO 4 FT IN THE AFTERNOON.
RAIN.
.TONIGHT...SE WIND 25 KT DIMINISHING TO 15 KT AFTER MIDNIGHT.
SEAS 4 FT SUBSIDING TO 2 FT AFTER MIDNIGHT. RAIN.
.MON...E WIND 10 KT INCREASING TO NE 15 KT IN THE AFTERNOON. 
SEAS 3 FT. RAIN.
.MON NIGHT...NE WIND 10 KT. SEAS 2 FT.
.TUE...SE WIND 10 KT. SEAS 2 FT.
.WED THROUGH THU...VARIABLE WIND LESS THAN 10 KT. SEAS 1 FT.



PKZ140-220100-
COOK INLET NORTH OF KALGIN ISLAND-
352 AM AKDT SUN AUG 21 2016

.TODAY...S WIND 20 KT. GUSTS TO 30 KT EARLY THIS MORNING. 
SEAS 6 FT SUBSIDING TO 4 FT IN THE AFTERNOON. RAIN.
.TONIGHT...SE WIND 20 KT BECOMING E 10 KT AFTER MIDNIGHT. 
SEAS 4 FT SUBSIDING. RAIN.
.MON...NE WIND 15 KT. SEAS 3 FT. RAIN.
.MON NIGHT...NE WIND 20 KT. SEAS 4 FT.
.TUE THROUGH WED...NE WIND 10 KT. SEAS 2 FT.
.THU...S WIND 10 KT. SEAS 2 F

## Putting it together

Given the caveat that my partner is only somewhat tech saavy, the goal is to take these functions from a set of scripts that need to be called manually, to a resource that's available when needed. This is especially important given potential issues with the Weather Underground API limit for the free-tier.